In [4]:
from pandas.io.parsers import read_csv
import numpy as np

np.random.seed(0) # REMOVE FOR FINAL SOLUTION


In [5]:
#Data Splitted.
def splitData(data):
    shuffled = data.copy()
    np.random.shuffle(shuffled)

    testCount = int(shuffled.shape[0]*1/3)
    trainCount = shuffled.shape[0]-testCount

    trainSet = shuffled[0:trainCount]
    testSet = shuffled[trainCount:]

    return trainSet, testSet

def separateInputOutput(dataset):
    outputData = dataset[:,3] # Output
    inputData = np.concatenate((dataset[:,0:3],dataset[:,4:7],dataset[:,8:10]),axis=1) # Inputs
    return inputData, outputData

#ML ALG
def initializeWeights(x,y):
    N = x.shape[0]


In [6]:
#Reading in data
dataframe = read_csv("beer.txt",delim_whitespace=True,header=None)

# Preparing Data
dataset = dataframe.values
trainSet, testSet = splitData(dataset)

testIn, testOut = separateInputOutput(testSet)
trainIn, trainOut = separateInputOutput(trainSet)

N =trainIn.shape[0]
Wi = 1/N

#running ml alg.
train(trainIn,trainOut) #ML ALG RAN HERE 


In [7]:
def getUniqueClassCount(listToCount):
    classes = []
    for item in listToCount:
        if item not in classes:
            classes.append(item)

    counts = []
    for itemClass in classes:
        counts.append(listToCount.count(itemClass))
    return counts

def getGiniLeaf(counts):
    total = np.sum(counts)
    giniLeaf = 1
    for count in counts:
        giniLeaf -= (count/total)**2
    return giniLeaf

def getGiniNode(sortedList, threshold):
        lessThan = []
        greaterThan = []

        for i in range(len(sortedList)):
            x = sortedList[i][0]
            # print(x, threshold)
            if x < threshold:
                lessThan.append(sortedList[i][1])
            else:
                greaterThan.append(sortedList[i][1])

        countsLT = getUniqueClassCount(lessThan)
        countsGT = getUniqueClassCount(greaterThan)
        giniLT = getGiniLeaf(countsLT)
        giniGT = getGiniLeaf(countsGT)
        # print(giniLT,giniGT)
        # print(np.sum(countsLT),np.sum(countsGT))
        totalCount = len(sortedList)
        # print(totalCount)
        giniNode = ((np.sum(countsLT)/totalCount)*giniLT) + ((np.sum(countsGT)/totalCount)*giniGT)
        # print(giniNode)
        return giniNode


In [40]:
def getSplitValue(trainIn, trainOut, attributeIndex):
    attribute = trainIn[:, attributeIndex].copy() # Output
    combined = []
    for j,sample in enumerate(attribute):
        combined.append((sample,trainOut[j]))
    attribute.sort()
    sortedList = [tuple for x in attribute for tuple in combined if tuple[0] == x]


    thresholds = []
    for j in range(len(attribute)-1):
        thresholds.append((attribute[j]+attribute[j+1])/2)

    giniNodes = []
    for threshold in thresholds:
        giniNodes.append(getGiniNode(sortedList, threshold))
    # print(giniNodes)
    minNodeGini = min(giniNodes)
    minGiniIndex = giniNodes.index(minNodeGini)
    splitVal = thresholds[minGiniIndex]
    return splitVal, minNodeGini
   

In [113]:
def train(trainIn,trainOut):
    ginis = []
    splitVals = []
    for i in range(trainIn.shape[1]):
        splitVal, minGini = getSplitValue(trainIn,trainOut, i)
        ginis.append(minGini)
        splitVals.append(splitVal)
        # print(splitVal, minGini)
    # print("Root is input data attribute", ginis.index(min(ginis)))
    branchData(testIn,testOut,ginis.index(min(ginis)),splitVals[ginis.index(min(ginis))]) #

Less Than  [38.09292035 0.261521856 1.7318181819999998 3.618461538 17.68 7.556157895
 14.76 81.21142857]


In [117]:
def branchData(X, y, splitAttributeIndex, splitValue):
    lessThan_X = []
    lessThan_y = []
    greaterThan_X = []
    greaterThan_y = []
    
    for i in range(len(X)):
        if X[i,splitAttributeIndex] < splitValue:
            lessThan_X.append(X[i,:])
            lessThan_y.append(y[i])
        else:
            greaterThan_X.append(X[i,:])
            greaterThan_y.append(y[i])